In [2]:
def get_cube_metadata(product_id, endpoint=None, timeout=30):
    """
    Retrieves metadata for a cube (table) from Statistics Canada WDS.

    Args:
        product_id (int or str): The product ID (PID) of the cube (e.g., 35100003).
        endpoint (str): Optional custom endpoint URL (defaults to the documented one).
        timeout (float): Timeout in seconds for the HTTP request.

    Returns:
        dict: JSON response object converted to Python dict.
    """
    if endpoint is None:
        endpoint = "https://www150.statcan.gc.ca/t1/wds/rest/getCubeMetadata"

    # Prepare payload as list of dicts, per documentation example. :contentReference[oaicite:2]{index=2}
    payload = [
        {
            "productId": str(product_id)
        }
    ]

    headers = {
        "Content-Type": "application/json"
    }

    try:
        response = requests.post(endpoint, headers=headers, json=payload, timeout=timeout)
        response.raise_for_status()
        data = response.json()

        # Check for status
        if data[0]["status"] != "SUCCESS":
            raise RuntimeError(f"Request returned non-SUCCESS status: {data}")
        
        return data[0]['object']
    
    except requests.RequestException as e:
        raise RuntimeError(f"HTTP request failed: {e}") from e
    
    except json.JSONDecodeError as e:
        raise RuntimeError(f"Failed to parse JSON response: {e}") from e

In [ ]:
import requests
import pandas as pd
from io import BytesIO
import urllib.parse
import json
from datetime import datetime, timedelta
from IPython.core.display import display, HTML


url = "https://www150.statcan.gc.ca/t1/wds/rest/getAllCubesList"
resp = requests.get(url)
tables = resp.json()

card_css = "border:1px solid black;margin:10px;padding:10px;background:white;max-width:200px;border-radius:3px;color:black;"
n = 2

count = 0

df = pd.DataFrame(tables)
df = df[[x for x in df.columns if x if 'Fr' not in x]]
df = df[df['archived'] == '2']

def get_table(id,n,enc,region_type=''):

    url = f'https://www150.statcan.gc.ca/t1/tbl1/en/dtl!downloadDbLoadingData-nonTraduit.action?pid={id}01&latestN={n}&startDate=&endDate=&csvLocale=en&selectedMembers={enc}&checkedLevels=0D1%2C0D1%2C0D2'
    print(url)
    df = pd.read_csv(url)
    if region_type != '':
        df = df[df.DGUID.str[6:9] == region_type]

    display(df)

def parse_dim(x):

    tmp = {}
    sel = []

    for i in x:

        tmp[i['dimensionNameEn']] = i['hasUOM']
        sel.append([1])
    
    return tmp,sel

archived_tables = df[df['archived'] == '1']
active_tables = df[df['archived'] == '2']

for p in tables:

    id = p['productId']
    endDate = datetime.fromisoformat(p['cubeEndDate'].replace("Z", "+00:00")).replace(tzinfo=None)

    if p['archived'] == '2':

        metadata = get_cube_metadata(id)
        num_dimensions = len(metadata['dimension'])
        data = metadata['dimension']

        tablename = p['cubeTitleEn']
        dim = p['dimensions']
        dim,selected = parse_dim(dim)

        raw = json.dumps(selected, separators=(',',':'))
        enc = urllib.parse.quote(raw)

        num_param = len(data)

        attributes = {}
        attributes['Table Name'] = tablename

        for i in range(0,num_param):

            sub_size = len(data[i]['member'])
            top_name = data[i]['dimensionNameEn']

            for j in range(0,sub_size):
                
                #print(data[i]['member'][j])

                name = data[i]['member'][j]['memberNameEn']
                classification = data[i]['member'][j]['classificationCode']

                if top_name in attributes:
                    attributes[top_name].append(name)
                else:
                    attributes[top_name] = [name]

        print(id,' - ',tablename,'\n',attributes)
        #get_table(id=id,n=n,enc=enc)#region_type='503')
        
        count += 1

        if count > 5:
            break
                



C:\Users\kylel\AppData\Local\Temp\ipykernel_19944\2154275600.py:7: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


10100002  -  Central government debt 
 {'Table Name': 'Central government debt', 'Geography': ['Canada'], 'Central government debt': ['A. Federal debt (accumulated deficit), (B - E)', 'B. Net debt, (C - D)', 'C. Liabilities, gross debt', 'Accounts payable and accrued liabilities', 'Interest-bearing debt', 'Unmatured debt', 'Market debt payable in Canadian currency', 'Marketable bonds payable in Canadian currency', 'Treasury bills', 'Retail debt', 'Other market debt payable in Canadian currency', 'Market debt payable in foreign currencies', 'Marketable bonds payable in foreign currencies', 'Medium-term notes', 'Canada bills', 'Cross currency swap revaluation', 'Unamortized discounts and premiums on market debt', 'Obligations related to capital leases and other unmatured debt', 'Pensions and other accounts', 'Foreign exchange accounts liabilities', 'Derivatives', 'D. Financial assets', 'Cash and accounts receivable', 'Foreign exchange accounts', 'Foreign exchange accounts assets', 'Deriv